In [1]:
from process_recording import *

from matplotlib import pyplot as plt

In [2]:
test_crop("./data/bahrain.mkv")

100%|██████████| 200/200 [00:01<00:00, 135.87it/s]


In [23]:
process_video("./data/imola.mkv", "./data/imola.csv", 6)

  0%|          | 278/190298 [00:08<1:36:29, 32.82it/s, time=1:13.307, date_text=11/5/2022]  


KeyboardInterrupt: 